# Pytorch Lab0

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

1.4.0
True


In [2]:
arguments={'batch_size':128,'epoch':20,'lr':0.01,'momentum':0.9,'no_cuda':False,'seed':1}
arguments['no_cuda']=(not arguments['no_cuda'])and torch.cuda.is_available()

## DataLoader

In [3]:
train_loader=torch.utils.data.DataLoader(datasets.MNIST('./MNIST_data',train=True,download=True,transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])),batch_size=arguments['batch_size'],shuffle=True,num_workers=2)
test_loader=torch.utils.data.DataLoader(datasets.MNIST('./MNIST_data',train=False,download=True,transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])),batch_size=arguments['batch_size'],shuffle=True,num_workers=2)

## Network

In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv1=nn.Conv2d(1,6,kernel_size=(5,5),stride=1,padding=0)
        self.conv2=nn.Conv2d(6,16,kernel_size=(5,5),stride=1,padding=0)
        self.ff1=nn.Linear(16*4*4,120)
        self.ff2=nn.Linear(120,84)
        self.ff3=nn.Linear(84,10)
    def forward(self,x):
        out=F.relu(self.conv1(x))
        out=F.max_pool2d(out,2)
        out=F.relu(self.conv2(out))
        out=F.max_pool2d(out,2)
        out=out.view(out.size()[0],-1)
        out=F.relu(self.ff1(out))
        out=F.relu(self.ff2(out))
        out=F.relu(self.ff3(out))
        return out

In [5]:
model=LeNet()
if not arguments['no_cuda']:
    device=torch.device('cuda')
    model.to(device)

## Loss function & Optimizer

In [6]:
Loss=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=arguments['lr'],momentum=arguments['momentum'])

In [7]:
def adjust_lr(optimizer,epoch):
    if epoch<10:
        lr=0.01
    elif epoch<20:
        lr=0.001
    else:
        lr=0.0001
    for param_group in optimizer.param_groups:
        param_group['lr']=lr
    return

## Train & Test

In [8]:
def train(epoch):
    model.train()
    adjust_lr(optimizer,epoch)
    for index,(data,target) in enumerate(train_loader):
        if not arguments['no_cuda']:
            data=data.to(device)
            target=target.to(device)
        predict=model(data)
        loss=Loss(predict,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Train epoch {epoch}, {index*len(data)}/{len(train_loader.dataset)}, loss:{loss:.4f}')

In [9]:
def test():
    model.eval()
    acc_loss=0
    correct=0
    for index,(data,target) in enumerate(test_loader):
        if not arguments['no_cuda']:
            data=data.to(device)
            target=target.to(device)
        with torch.no_grad():
            predict=model(data)
        acc_loss+=Loss(predict,target)
        correct+=predict.data.max(1)[1].eq(target.data).cpu().sum()
    print(f'Test acc:{100.*correct/len(test_loader.dataset):.2f}%, avg loss:{acc_loss/len(test_loader):.4f}')

## Run

In [10]:
for epoch in range(1,arguments['epoch']+1):
    train(epoch)
test()
torch.save(model.state_dict(),'./LeNet.pt')

Train epoch 1, 0/60000, loss:2.3044
Train epoch 1, 128/60000, loss:2.3040
Train epoch 1, 256/60000, loss:2.3071
Train epoch 1, 384/60000, loss:2.3030
Train epoch 1, 512/60000, loss:2.2974
Train epoch 1, 640/60000, loss:2.3037
Train epoch 1, 768/60000, loss:2.2981
Train epoch 1, 896/60000, loss:2.3034
Train epoch 1, 1024/60000, loss:2.2944
Train epoch 1, 1152/60000, loss:2.3023
Train epoch 1, 1280/60000, loss:2.3063
Train epoch 1, 1408/60000, loss:2.2988
Train epoch 1, 1536/60000, loss:2.3001
Train epoch 1, 1664/60000, loss:2.3001
Train epoch 1, 1792/60000, loss:2.2894
Train epoch 1, 1920/60000, loss:2.2979
Train epoch 1, 2048/60000, loss:2.3023
Train epoch 1, 2176/60000, loss:2.3044
Train epoch 1, 2304/60000, loss:2.3015
Train epoch 1, 2432/60000, loss:2.2967
Train epoch 1, 2560/60000, loss:2.3050
Train epoch 1, 2688/60000, loss:2.2921
Train epoch 1, 2816/60000, loss:2.2928
Train epoch 1, 2944/60000, loss:2.2908
Train epoch 1, 3072/60000, loss:2.2998
Train epoch 1, 3200/60000, loss:2.2

Train epoch 1, 26880/60000, loss:0.7430
Train epoch 1, 27008/60000, loss:0.6706
Train epoch 1, 27136/60000, loss:0.7214
Train epoch 1, 27264/60000, loss:0.6614
Train epoch 1, 27392/60000, loss:0.7166
Train epoch 1, 27520/60000, loss:0.7789
Train epoch 1, 27648/60000, loss:0.6637
Train epoch 1, 27776/60000, loss:0.6324
Train epoch 1, 27904/60000, loss:0.6611
Train epoch 1, 28032/60000, loss:0.7190
Train epoch 1, 28160/60000, loss:0.7729
Train epoch 1, 28288/60000, loss:0.7416
Train epoch 1, 28416/60000, loss:0.6317
Train epoch 1, 28544/60000, loss:0.8323
Train epoch 1, 28672/60000, loss:0.6732
Train epoch 1, 28800/60000, loss:0.6910
Train epoch 1, 28928/60000, loss:0.6933
Train epoch 1, 29056/60000, loss:0.6657
Train epoch 1, 29184/60000, loss:0.7341
Train epoch 1, 29312/60000, loss:0.5723
Train epoch 1, 29440/60000, loss:0.6201
Train epoch 1, 29568/60000, loss:0.7838
Train epoch 1, 29696/60000, loss:0.7392
Train epoch 1, 29824/60000, loss:0.6064
Train epoch 1, 29952/60000, loss:0.5726


Train epoch 1, 54144/60000, loss:0.4882
Train epoch 1, 54272/60000, loss:0.5455
Train epoch 1, 54400/60000, loss:0.4692
Train epoch 1, 54528/60000, loss:0.5280
Train epoch 1, 54656/60000, loss:0.5693
Train epoch 1, 54784/60000, loss:0.5897
Train epoch 1, 54912/60000, loss:0.6139
Train epoch 1, 55040/60000, loss:0.5339
Train epoch 1, 55168/60000, loss:0.4895
Train epoch 1, 55296/60000, loss:0.5200
Train epoch 1, 55424/60000, loss:0.6669
Train epoch 1, 55552/60000, loss:0.5297
Train epoch 1, 55680/60000, loss:0.4266
Train epoch 1, 55808/60000, loss:0.4551
Train epoch 1, 55936/60000, loss:0.5565
Train epoch 1, 56064/60000, loss:0.5694
Train epoch 1, 56192/60000, loss:0.5832
Train epoch 1, 56320/60000, loss:0.5898
Train epoch 1, 56448/60000, loss:0.7016
Train epoch 1, 56576/60000, loss:0.4596
Train epoch 1, 56704/60000, loss:0.6299
Train epoch 1, 56832/60000, loss:0.6197
Train epoch 1, 56960/60000, loss:0.4917
Train epoch 1, 57088/60000, loss:0.5715
Train epoch 1, 57216/60000, loss:0.5535


Train epoch 2, 21248/60000, loss:0.2925
Train epoch 2, 21376/60000, loss:0.1110
Train epoch 2, 21504/60000, loss:0.2228
Train epoch 2, 21632/60000, loss:0.2659
Train epoch 2, 21760/60000, loss:0.2074
Train epoch 2, 21888/60000, loss:0.2121
Train epoch 2, 22016/60000, loss:0.3326
Train epoch 2, 22144/60000, loss:0.2507
Train epoch 2, 22272/60000, loss:0.3262
Train epoch 2, 22400/60000, loss:0.4618
Train epoch 2, 22528/60000, loss:0.4315
Train epoch 2, 22656/60000, loss:0.3263
Train epoch 2, 22784/60000, loss:0.3422
Train epoch 2, 22912/60000, loss:0.3014
Train epoch 2, 23040/60000, loss:0.2909
Train epoch 2, 23168/60000, loss:0.3401
Train epoch 2, 23296/60000, loss:0.2493
Train epoch 2, 23424/60000, loss:0.3020
Train epoch 2, 23552/60000, loss:0.2337
Train epoch 2, 23680/60000, loss:0.4351
Train epoch 2, 23808/60000, loss:0.2611
Train epoch 2, 23936/60000, loss:0.3922
Train epoch 2, 24064/60000, loss:0.4623
Train epoch 2, 24192/60000, loss:0.2739
Train epoch 2, 24320/60000, loss:0.2204


Train epoch 2, 47744/60000, loss:0.2969
Train epoch 2, 47872/60000, loss:0.3668
Train epoch 2, 48000/60000, loss:0.4017
Train epoch 2, 48128/60000, loss:0.3177
Train epoch 2, 48256/60000, loss:0.2350
Train epoch 2, 48384/60000, loss:0.2899
Train epoch 2, 48512/60000, loss:0.2613
Train epoch 2, 48640/60000, loss:0.3241
Train epoch 2, 48768/60000, loss:0.2869
Train epoch 2, 48896/60000, loss:0.2553
Train epoch 2, 49024/60000, loss:0.3225
Train epoch 2, 49152/60000, loss:0.2624
Train epoch 2, 49280/60000, loss:0.3297
Train epoch 2, 49408/60000, loss:0.3481
Train epoch 2, 49536/60000, loss:0.3062
Train epoch 2, 49664/60000, loss:0.2361
Train epoch 2, 49792/60000, loss:0.3142
Train epoch 2, 49920/60000, loss:0.3429
Train epoch 2, 50048/60000, loss:0.4106
Train epoch 2, 50176/60000, loss:0.3940
Train epoch 2, 50304/60000, loss:0.3423
Train epoch 2, 50432/60000, loss:0.2210
Train epoch 2, 50560/60000, loss:0.3112
Train epoch 2, 50688/60000, loss:0.3409
Train epoch 2, 50816/60000, loss:0.2729


...

Train epoch 19, 8320/60000, loss:0.1303
Train epoch 19, 8448/60000, loss:0.1358
Train epoch 19, 8576/60000, loss:0.2430
Train epoch 19, 8704/60000, loss:0.2002
Train epoch 19, 8832/60000, loss:0.2374
Train epoch 19, 8960/60000, loss:0.1822
Train epoch 19, 9088/60000, loss:0.2604
Train epoch 19, 9216/60000, loss:0.3267
Train epoch 19, 9344/60000, loss:0.2687
Train epoch 19, 9472/60000, loss:0.1481
Train epoch 19, 9600/60000, loss:0.3346
Train epoch 19, 9728/60000, loss:0.2559
Train epoch 19, 9856/60000, loss:0.2352
Train epoch 19, 9984/60000, loss:0.2884
Train epoch 19, 10112/60000, loss:0.1890
Train epoch 19, 10240/60000, loss:0.1501
Train epoch 19, 10368/60000, loss:0.2185
Train epoch 19, 10496/60000, loss:0.2714
Train epoch 19, 10624/60000, loss:0.2663
Train epoch 19, 10752/60000, loss:0.2097
Train epoch 19, 10880/60000, loss:0.2401
Train epoch 19, 11008/60000, loss:0.1824
Train epoch 19, 11136/60000, loss:0.2775
Train epoch 19, 11264/60000, loss:0.2613
Train epoch 19, 11392/60000, l

Train epoch 19, 34816/60000, loss:0.2752
Train epoch 19, 34944/60000, loss:0.2904
Train epoch 19, 35072/60000, loss:0.1513
Train epoch 19, 35200/60000, loss:0.3311
Train epoch 19, 35328/60000, loss:0.2500
Train epoch 19, 35456/60000, loss:0.1802
Train epoch 19, 35584/60000, loss:0.1659
Train epoch 19, 35712/60000, loss:0.2346
Train epoch 19, 35840/60000, loss:0.2565
Train epoch 19, 35968/60000, loss:0.2065
Train epoch 19, 36096/60000, loss:0.1281
Train epoch 19, 36224/60000, loss:0.1520
Train epoch 19, 36352/60000, loss:0.1729
Train epoch 19, 36480/60000, loss:0.2251
Train epoch 19, 36608/60000, loss:0.3108
Train epoch 19, 36736/60000, loss:0.2176
Train epoch 19, 36864/60000, loss:0.1319
Train epoch 19, 36992/60000, loss:0.2604
Train epoch 19, 37120/60000, loss:0.2886
Train epoch 19, 37248/60000, loss:0.1568
Train epoch 19, 37376/60000, loss:0.2903
Train epoch 19, 37504/60000, loss:0.2081
Train epoch 19, 37632/60000, loss:0.1502
Train epoch 19, 37760/60000, loss:0.2914
Train epoch 19, 

Train epoch 20, 1152/60000, loss:0.2265
Train epoch 20, 1280/60000, loss:0.1464
Train epoch 20, 1408/60000, loss:0.1990
Train epoch 20, 1536/60000, loss:0.1818
Train epoch 20, 1664/60000, loss:0.1728
Train epoch 20, 1792/60000, loss:0.3411
Train epoch 20, 1920/60000, loss:0.1862
Train epoch 20, 2048/60000, loss:0.1444
Train epoch 20, 2176/60000, loss:0.1450
Train epoch 20, 2304/60000, loss:0.2768
Train epoch 20, 2432/60000, loss:0.1494
Train epoch 20, 2560/60000, loss:0.2508
Train epoch 20, 2688/60000, loss:0.2396
Train epoch 20, 2816/60000, loss:0.2550
Train epoch 20, 2944/60000, loss:0.2156
Train epoch 20, 3072/60000, loss:0.2542
Train epoch 20, 3200/60000, loss:0.2069
Train epoch 20, 3328/60000, loss:0.3790
Train epoch 20, 3456/60000, loss:0.1815
Train epoch 20, 3584/60000, loss:0.1631
Train epoch 20, 3712/60000, loss:0.1634
Train epoch 20, 3840/60000, loss:0.2572
Train epoch 20, 3968/60000, loss:0.3010
Train epoch 20, 4096/60000, loss:0.3273
Train epoch 20, 4224/60000, loss:0.2563


Train epoch 20, 27264/60000, loss:0.2486
Train epoch 20, 27392/60000, loss:0.1478
Train epoch 20, 27520/60000, loss:0.1505
Train epoch 20, 27648/60000, loss:0.1464
Train epoch 20, 27776/60000, loss:0.0917
Train epoch 20, 27904/60000, loss:0.3423
Train epoch 20, 28032/60000, loss:0.1295
Train epoch 20, 28160/60000, loss:0.3094
Train epoch 20, 28288/60000, loss:0.2017
Train epoch 20, 28416/60000, loss:0.2423
Train epoch 20, 28544/60000, loss:0.1826
Train epoch 20, 28672/60000, loss:0.3256
Train epoch 20, 28800/60000, loss:0.2183
Train epoch 20, 28928/60000, loss:0.1834
Train epoch 20, 29056/60000, loss:0.2216
Train epoch 20, 29184/60000, loss:0.3063
Train epoch 20, 29312/60000, loss:0.3481
Train epoch 20, 29440/60000, loss:0.1881
Train epoch 20, 29568/60000, loss:0.2713
Train epoch 20, 29696/60000, loss:0.1849
Train epoch 20, 29824/60000, loss:0.1645
Train epoch 20, 29952/60000, loss:0.2073
Train epoch 20, 30080/60000, loss:0.2178
Train epoch 20, 30208/60000, loss:0.2760
Train epoch 20, 

Train epoch 20, 53248/60000, loss:0.2888
Train epoch 20, 53376/60000, loss:0.3066
Train epoch 20, 53504/60000, loss:0.2757
Train epoch 20, 53632/60000, loss:0.2908
Train epoch 20, 53760/60000, loss:0.1099
Train epoch 20, 53888/60000, loss:0.2710
Train epoch 20, 54016/60000, loss:0.1265
Train epoch 20, 54144/60000, loss:0.2208
Train epoch 20, 54272/60000, loss:0.2418
Train epoch 20, 54400/60000, loss:0.2360
Train epoch 20, 54528/60000, loss:0.2364
Train epoch 20, 54656/60000, loss:0.2764
Train epoch 20, 54784/60000, loss:0.2020
Train epoch 20, 54912/60000, loss:0.1654
Train epoch 20, 55040/60000, loss:0.2099
Train epoch 20, 55168/60000, loss:0.2931
Train epoch 20, 55296/60000, loss:0.2354
Train epoch 20, 55424/60000, loss:0.1512
Train epoch 20, 55552/60000, loss:0.1660
Train epoch 20, 55680/60000, loss:0.1998
Train epoch 20, 55808/60000, loss:0.2426
Train epoch 20, 55936/60000, loss:0.2712
Train epoch 20, 56064/60000, loss:0.2705
Train epoch 20, 56192/60000, loss:0.1998
Train epoch 20, 